<a href="https://colab.research.google.com/github/NicholasCharlesYu/LightweightFineTuningModel/blob/main/LightweightFineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique:
* Model:
* Evaluation approach:
* Fine-tuning dataset:

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [39]:
! pip install -q "datasets==2.15.0" "transformers==4.30.2" "peft" "accelerate"
!pip install -q transformers[torch]
import torch
from datasets import load_dataset

ds = load_dataset("cornell-movie-review-data/rotten_tomatoes")

In [40]:
import numpy as np

In [41]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

In [42]:
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
model.config.pad_token_id = tokenizer.pad_token_id

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [43]:
tokenized_ds = ds.map(lambda examples: tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128), batched=True)
tokenized_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
train_data = tokenized_ds["train"]
test_data = tokenized_ds["test"]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [44]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
    ),
    eval_dataset=test_data,
    data_collator=data_collator,
)

trainer.evaluate()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 7.77259635925293,
 'eval_runtime': 8.5787,
 'eval_samples_per_second': 124.261,
 'eval_steps_per_second': 15.62}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [45]:
from peft import LoraConfig, get_peft_model
config = LoraConfig(
    r=4,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    )

In [46]:
lora_model = get_peft_model(model, config)
lora_model.print_trainable_parameters()

trainable params: 147,456 || all params: 124,588,800 || trainable%: 0.1184


/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1119: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [51]:
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
    ),
    eval_dataset=test_data,
    train_dataset=train_data,
    data_collator=data_collator,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.834600,0.699921
2,0.588200,0.528245
3,0.539000,0.473971


TrainOutput(global_step=3201, training_loss=0.9345421497615491, metrics={'train_runtime': 535.1388, 'train_samples_per_second': 47.819, 'train_steps_per_second': 5.982, 'total_flos': 1674543932375040.0, 'train_loss': 0.9345421497615491, 'epoch': 3.0})

In [52]:
lora_model.save_pretrained("gpt-lora")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [61]:
from peft import AutoPeftModelForCausalLM
peft_model = AutoPeftModelForCausalLM.from_pretrained("gpt-lora")
peft_model.config.pad_token_id = tokenizer.pad_token_id
trainer.model = peft_model


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [66]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
peft_model = peft_model.to(device)
trainer.evaluate()

{'eval_loss': 0.71636962890625,
 'eval_runtime': 9.7487,
 'eval_samples_per_second': 109.348,
 'eval_steps_per_second': 13.745,
 'epoch': 3.0}